# Corporate Actions - Smashing the data sets and visualizing the results

First we define some global constants to keep us in line

In [1]:
# Types: merger, spinoff, split, dividend
ca_types = ['merger','spinoff', 'split', 'dividend']

# Events: effective (announcement will date effect), record (last date to ensure entitlement), declaration (announced), payable (payout), ex (first date after which no entitlement)
ca_events = ['effective', 'record', 'declaration', 'payable', 'ex']

Import the ticker data. For this particular piece, I suspect we will want to use unadjusted values

In [2]:
import pandas as pd

def parse_data(json_data: str):
    df = pd.read_json(json_data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_values(by='date', inplace=True)
    df['Close'] = df['close']
    df['Open'] = df['open']
    df['High'] = df['high']
    df['Low'] = df['low']
    df['Volume'] = df['volume']
    return df

Add the corporate actions indicators. We create one new column for each type and event with the Closing price as the value.

In [3]:
def add_ca_data(df: pd.DataFrame, json_data: str):
    ca_df = pd.read_json(json_data)
    ca_df['effective_date'] = pd.to_datetime(ca_df['effective_date'])
    ca_df['record_date'] = pd.to_datetime(ca_df['record_date'])
    ca_df['declaration_date'] = pd.to_datetime(ca_df['declaration_date'])
    ca_df['payable_date'] = pd.to_datetime(ca_df['payable_date'])
    ca_df['ex_date'] = pd.to_datetime(ca_df['ex_date'])

    for type in ca_types:
        for event in ca_events:
            df[f'ca_{type}_{event}'] = None

    symbol = df['symbol'][0]

    ca_df_filtered = ca_df.loc[ca_df['initiating_symbol'] == symbol]

    for index, row in ca_df_filtered.iterrows():
        df.loc[df.index == row['effective_date'], [f"ca_{row['ca_type']}_effective"]] = df.loc[df.index == row['effective_date']]['Close']
        df.loc[df.index == row['record_date'], [f"ca_{row['ca_type']}_record"]] = df.loc[df.index == row['record_date']]['Close']
        df.loc[df.index == row['declaration_date'], [f"ca_{row['ca_type']}_declaration"]] = df.loc[df.index == row['declaration_date']]['Close']
        df.loc[df.index == row['payable_date'], [f"ca_{row['ca_type']}_payable"]] = df.loc[df.index == row['payable_date']]['Close']
        df.loc[df.index == row['ex_date'], [f"ca_{row['ca_type']}_ex"]] = df.loc[df.index == row['ex_date']]['Close']

    return df


    

Plotting....

In [27]:
import plotly.offline as pyo
import plotly.graph_objects as go

def create_plot(df: pd.DataFrame):

    dfpl = df.last('90D').copy()

    data = [go.Candlestick(x=dfpl.index,
                           open=dfpl['Open'],               
                           high=dfpl['High'],
                           low=dfpl['Low'],
                           close=dfpl['Close'], yaxis='y1')]

    for type in ca_types:
      for event in ca_events:
         data = data + [go.Scatter(x=dfpl.index, y=dfpl[f'ca_{type}_{event}'], mode="markers", marker=dict(size=15, color="darkorange", symbol="triangle-down"), name=f"{type} - {event}", hovertemplate=f'{type} - {event}:' + '%{y:.2f}', yaxis='y1')]

    layout = go.Layout(title=f"Corporate actions for {df['symbol'][0]} ", yaxis_title='Price', dragmode='pan',
                       xaxis=dict(rangeslider=dict(visible=False)),
                       yaxis=dict(fixedrange=False, domain=[0,1], anchor='y1'))

    fig = go.Figure(data=data, layout=layout)

    pyo.plot(fig, filename=f"ca_{df['symbol'][0]}.html", auto_open=True)

Trying it all out

In [33]:
f = open('../development_assets/cas_filtered.json', 'r').read()
# stockdata = open('../development_assets/AAPL_10y.json').read()
# stockdata = open('../development_assets/V_10y.json').read()
# stockdata = open('../development_assets/AMZN_10y.json').read()
# stockdata = open('../development_assets/JNJ_10y.json').read()
stockdata = open('../development_assets/MSFT_10y.json').read()
df = parse_data(stockdata)
add_ca_data(df, f)
create_plot(df)
# df = add_ca_points(df)